In [1]:
# for generating the feature vector
import json
from nltk import ngrams
import nltk
from collections import Counter

# for NN using Keras
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

/Users/abhishekagrawal/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def extract_headline_category(category_list):
    headline_category = []
    try:
        input_file = open('./News_Category_Dataset.json')
        input_data = input_file.readlines()
        input_file.close()
        for json_object in input_data:
            data = json.loads(json_object)
            category = data['category'].upper()
            if category in category_list:
                headline_category.append((data['headline'], data['category']))
        return headline_category
    except IOError:
        print("ERROR : IO ERROR occurred while opening file")
        exit(0)

In [3]:
category_list = ['Business', 'Comedy','Sports', 'Crime', 'Religion']
category_list = [i.upper() for i in category_list]
headlines_and_category = extract_headline_category(category_list)

In [4]:
def get_n_grams(dataset:'headline, category', n:'n gram value', k:'return top k n-grams'):
    n_grams_list=[]
    for headline, category in dataset:
        tokenize = nltk.word_tokenize(headline)
        n_gram = nltk.ngrams(tokenize, n)
        n_grams_list.extend(n_gram)
    print(len(n_grams_list))
    top_k = Counter(n_grams_list).most_common(k)
    most_frequent=[i[0] for i in top_k]
    return most_frequent

In [5]:
unigrams_dict = get_n_grams(headlines_and_category, 1, 500)
bigrams_dict = get_n_grams(headlines_and_category, 2, 300)
trigrams_dict = get_n_grams(headlines_and_category, 3, 200)

187445


/Users/abhishekagrawal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: generator 'ngrams' raised StopIteration
  


169606
151792


In [6]:
pos_list = list({'CD', 'CC', 'RP', 'NNPS', 'IN', ',', '$', 'FW', 'RBR', 'JJ', "''", ')', 'VBD', 'VBP', 'POS', ':', 'NNS', '#', 'PRP', '(', 'VBN', 'PDT', 'JJS', 'VBG', 'PRP$', 'RBS', 'LS', '.', 'EX', 'NN', '``', 'DT', 'RB', 'WDT', 'VB', 'UH', 'TO', 'JJR', 'VBZ', 'MD', 'NNP', 'WP', 'WRB'})

In [7]:
def generate_features(dataset, unigrams_dict, bigrams_dict, trigrams_dict, pos_list, category_list):
    X = []
    y = []
    for headline, category in dataset:
        text = nltk.word_tokenize(headline)
        unigrams = nltk.ngrams(text, 1)
        bigrams = nltk.ngrams(text, 2)
        trigrams = nltk.ngrams(text, 3)
#         list_of_ngrams_dict = [unigrams_dict]
#         list_of_ngrams_sentences = [unigrams]
        list_of_ngrams_dict = [unigrams_dict, bigrams_dict, trigrams_dict]
        list_of_ngrams_sentences = [unigrams, bigrams, trigrams]
        temp_sentence = []
        for i, ngram in enumerate(list_of_ngrams_dict):
            temp_ngram = [0]*len(ngram)
            for word in list_of_ngrams_sentences[i]:
                if word in ngram:
                    temp_ngram[ngram.index(word)]+=1
            temp_sentence.extend(temp_ngram)
        
    
#         temp_pos = [0]*len(pos_list)
#         for word, tag in nltk.pos_tag(text):
#             temp_pos[pos_list.index(tag)]+=1
#         temp_sentence.extend(temp_pos)
        X.append(temp_sentence)
        y.append(category_list.index(category))
    return (X,y)

In [8]:
X, y = generate_features(headlines_and_category, unigrams_dict, bigrams_dict, trigrams_dict, pos_list, category_list)

/Users/abhishekagrawal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: generator 'ngrams' raised StopIteration
  app.launch_new_instance()


In [9]:
y_temp = []
for i in y:
    temp = [0, 0, 0, 0, 0]
    temp[i] = 1
    y_temp.append(temp)
y = y_temp

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

in_features = len(X_train[0])
out_features = len(category_list)

In [10]:
# Keras
model = Sequential()
model.add(Dense(64, input_dim=in_features, activation='relu'))
model.add(Dense(16, input_dim=64, activation='relu'))
model.add(Dense(out_features, activation='softmax'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['acc'])
model.fit(np.array(X_train), np.array(y_train), epochs=50, verbose=1)

Epoch 1/50
14272/14272 [==============================] - 2s 109us/step - loss: 0.1044 - acc: 0.6003
Epoch 2/50
14272/14272 [==============================] - 1s 85us/step - loss: 0.0719 - acc: 0.7340
Epoch 3/50
14272/14272 [==============================] - 1s 89us/step - loss: 0.0664 - acc: 0.7549
Epoch 4/50
14272/14272 [==============================] - 1s 94us/step - loss: 0.0630 - acc: 0.7705
Epoch 5/50
14272/14272 [==============================] - 2s 135us/step - loss: 0.0598 - acc: 0.7848
Epoch 6/50
14272/14272 [==============================] - 2s 106us/step - loss: 0.0571 - acc: 0.7958
Epoch 7/50
14272/14272 [==============================] - 2s 109us/step - loss: 0.0539 - acc: 0.8103
Epoch 8/50
14272/14272 [==============================] - 1s 93us/step - loss: 0.0508 - acc: 0.8246
Epoch 9/50
14272/14272 [==============================] - 2s 114us/step - loss: 0.0477 - acc: 0.8394
Epoch 10/50
14272/14272 [==============================] - 1s 102us/step - loss: 0.0449 - acc: 

In [11]:
pred = model.predict(np.array(X_test)).round()
print(accuracy_score(np.array(y_test), pred))

0.6615298402913982
